# Sistema de Recomendación de Cultivos
## Entrega 1: EDA completo

**Machine Learning**

**Profesor:** Marco Terán  
**Fecha:** 2025/09/30

**Integrantes**

- Daniel Alejandro Garcia Zuluaica
- Edward Alejandro Rayo Cortés
- Elizabeth Toro Chalarca

___

## 📋 Tabla de Contenidos

1. **Configuración del entorno**
2. **Comprensión del Negocio**
3. **Obtención y Comprensión de Datos**
4. **Análisis Exploratorio (EDA)**

---

In [ ]:
print("¡Bienvenidos al primer notebook!")

---

## 1. Configuración del Entorno

### ¿Por qué importan las versiones?

En ML, la reproducibilidad es crucial. Imagina que tu modelo funciona perfectamente en tu computadora pero falla en producción. La causa más común: diferentes versiones de librerías.

**Regla de oro**: Siempre documenta y verifica las versiones de tus dependencias.

### Librerías que usaremos

- **NumPy**: El motor matemático de Python. Maneja arrays y operaciones numéricas eficientemente
- **Pandas**: Como Excel con superpoderes. Organiza datos en DataFrames (tablas)
- **Matplotlib/Seaborn**: Nuestros artistas. Crean visualizaciones profesionales
- **Scikit-learn**: La navaja suiza del ML. Contiene algoritmos, métricas y utilidades

### Configuración visual

Los defaults de matplotlib no son los más bonitos. Vamos a configurar:
- Estilo consistente para todos los gráficos
- Tamaños legibles
- Colores agradables
- Formato de números apropiado

In [ ]:
# Configuración inicial del entorno
import sys
import warnings
warnings.filterwarnings('ignore')

# Verificar versión de Python
assert sys.version_info >= (3, 7), "Este notebook requiere Python 3.7 o superior"

print(f"✅ Python {sys.version_info.major}.{sys.version_info.minor} instalado correctamente")

## Importando Librerias

Descargarems las librerias necesarias para utilizar el modelo, esto solo se ejecuta una vez

In [ ]:
pip install -r ../requirements.txt

### Validación de librerias

Se verifica que las librerias que vamos a usar estén completamente descargadas


- **NumPy**
- **Pandas**
- **Matplotlib/Seaborn**
- **Scikit-learn**

In [ ]:
# Importar librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12
sns.set_palette("husl")

# Configuración de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✅ Librerías importadas correctamente")

#### Verificar versiones de librerías críticas
Se valida que tengamos la versión mínima de las librerias

In [ ]:

from packaging import version
import sklearn

assert version.parse(sklearn.__version__) >= version.parse("1.0.1"), "Requiere scikit-learn >= 1.0.1"
print(f"✅ scikit-learn {sklearn.__version__} instalado")

## 💼 2. Fase 1: Comprensión del Negocio <a name="business_understanding"></a>

### El problema de seleccionar un cultivo invalido

**Contexto**: Imagina que eres un agricultor y quieres determinar cuál sería el mejor cultivo para sembrar sin verte afectado por factores de perdidas económicas, mal uso de los recursos y/o degradaciones ambientales.

La agricultura es fundamental para la seguridad alimentaria y el desarrollo económico. Los agricultores a menudo enfrentan desafíos al seleccionar el cultivo adecuado 
para su tierra según las condiciones del suelo y el ambiente. Una mala selección de cultivos puede llevar a:
 
**Problema actual**:
- Rendimientos reducidos: Los cultivos inadecuados pueden producir 30–50% menos que cultivos óptimos para las mismas condiciones [1].
- Pérdidas económicas: Estimadas en $1,000–$5,000 USD por hectárea por temporada en cultivos no optimizados [2].
- Uso ineficiente de recursos: Hasta 40% de desperdicio en agua y fertilizantes cuando el cultivo no coincide con las condiciones del suelo [3].
- Degradación ambiental: Agotamiento de nutrientes del suelo y erosión por prácticas agrícolas inadecuadas [4].

**Solución propuesta**: Determinar el conjunto de características y el algoritmo de Machine Learning que ofrezca el mejor desempeño en la predicción del cultivo más adecuado, considerando variables que representan los nutrientes del suelo (N, P, K) y condiciones agroclimáticas (pH, temperatura, humedad y precipitación).

### Definiendo el éxito

**Métrica de negocio**: 
- **Incremento en rendimiento**: Aumento del 15-20% en productividad de cultivos
	- Justificación: Estudios recientes han demostrado que los sistemas de recomendación de cultivos basados en Machine Learning pueden mejorar el rendimiento entre un 15% y 30% respecto a métodos tradicionales [2][3][5].
	- Relevancia: Este aumento representa una mejora tangible en la productividad agrícola, lo que incentiva la adopción del sistema por parte de los agricultores.

- **Reducción de costos**: Disminución del 25% en pérdidas por selección incorrecta
	- Justificación: La selección incorrecta de cultivos puede generar pérdidas de hasta $5,000 USD por hectárea [2]. Al recomendar cultivos más adecuados, se reduce el uso innecesario de fertilizantes, agua y otros insumos, lo que se traduce en una disminución directa de los costos operativos.
	- Relevancia: Esta métrica es crítica para demostrar el valor económico del sistema, especialmente en regiones con recursos limitados.

- **ROI**: Retorno de inversión positivo en 2 temporadas de cultivo
	- Justificación: Si el sistema logra aumentar el rendimiento y reducir los costos en una sola temporada, el agricultor puede recuperar la inversión en tecnología, capacitación o infraestructura en menos de dos ciclos agrícolas.
	- Relevancia: Un ROI rápido es clave para la sostenibilidad del proyecto y para convencer a financiadores o instituciones agrícolas.

- **Adopción**: Al menos 80% de recomendaciones implementadas por agricultores piloto
	- Justificación: La tasa de adopción refleja la aceptación del sistema en condiciones reales. Estudios previos muestran que los agricultores tienden a adoptar tecnologías que demuestran beneficios claros en campo [5][6].
	- Relevancia: Esta métrica valida la usabilidad y confianza en el sistema, y es esencial para escalar el proyecto a nivel regional o nacional.

**Métrica técnica**:
- **Accuracy >= 95%**: 
	- Justificación: En un problema de clasificación multiclase con 22 clases, un modelo aleatorio tendría una precisión de apenas ~4.5%. Alcanzar una accuracy ≥95% indica que el modelo ha aprendido patrones significativos y útiles para la predicción [2][3].
	- Evidencia: Modelos recientes como redes neuronales profundas y transformers (GPT-2) han alcanzado precisiones superiores al 97% en tareas similares de recomendación de cultivos [2][3].
	- Relevancia: Una alta accuracy genera confianza en los agricultores y valida el uso del sistema en campo [5].

- **Precision y Recall ≥ 90% por clase**
	- Justificación:
		- Precision alta: Evita falsos positivos, es decir, recomendar cultivos inadecuados que pueden generar pérdidas económicas significativas (~$3,000 USD/ha) [2].
		- Recall alto: Evita falsos negativos, asegurando que no se omitan cultivos viables que podrían mejorar el rendimiento.
	- Evidencia: Estudios como el de Reddy et al. (2024) y Bhola & Kumar (2024) muestran que los modelos bien calibrados pueden mantener estos valores por clase en problemas agrícolas complejos [2][6].
	- Relevancia: Estas métricas son críticas para decisiones agrícolas, donde cada recomendación tiene un impacto económico directo.

- **F1-Score ≥ 90%**
	El F1-Score combina precisión y exhaustividad, lo que permite evaluar el rendimiento del modelo de forma equilibrada, especialmente en clases con menor representación.
	En sistemas de recomendación agrícola, el F1-Score ≥90% ha sido usado como estándar para validar modelos antes de su implementación en campo [4][6].

**Nota sobre el tipo de problema**:
Este es un problema de **clasificación multiclase**, NO de regresión. Por tanto, MAE (Error Absoluto Medio) no es la métrica apropiada. MAE se usa para predecir valores continuos (ej: precio, cantidad), pero aquí predecimos categorías (tipos de cultivo).

### Preguntas críticas antes de empezar

**¿Realmente necesitamos ML?** Sí, por las siguientes razones:
- Complejidad del problema: Con 7 variables continuas y 22 clases, existen más de 10⁶ combinaciones posibles. Es impracticable crear reglas manuales [2].
- Interacciones no lineales: Las relaciones entre N-P-K, clima y pH son complejas y no evidentes. ML puede capturar estos patrones de forma más precisa que los métodos tradicionales [5].
- Escalabilidad: Un sistema ML puede analizar miles de casos en segundos, mientras que un agrónomo tardaría horas en evaluar cada uno manualmente.
- Evidencia científica: Estudios recientes muestran que los sistemas ML para recomendación de cultivos superan en 15–30% a métodos basados en reglas o expertos humanos [2][3][5][6].

Alternativas consideradas y descartadas:

Tablas de decisión manual: No escalables, no capturan interacciones complejas.
Consulta a expertos agrícolas: Costoso ($100–200 USD/consulta), subjetivo, no disponible 24/7.
Sistemas basados en reglas simples: Accuracy <70% en evaluaciones previas [5].

**¿Qué pasa si el modelo falla?** Análisis de riesgos y mitigación:

**Escenario 1**: Falso Positivo (recomienda cultivo inadecuado)
	- Impacto: Pérdida de inversión de temporada (~$3,000 USD/ha).
	Probabilidad con modelo ≥95% accuracy: ~2–3% de casos.
	- Mitigación:Mostrar solo recomendaciones con >85% de probabilidad.
	Validación humana por agrónomo en casos de baja confianza.
	Implementación piloto en ≤20% del área antes de escalar.

**Escenario 2**: Falso Negativo (no recomienda cultivo adecuado)
	- Impacto: Costo de oportunidad (rendimiento subóptimo). Probabilidad: ~5% de casos.
	- Mitigación: Mostrar top-3 recomendaciones con probabilidades. Permitir retroalimentación del agricultor para mejorar el sistema.

**Escenario 3**: Falla total del sistema
	- Impacto: Reversión a métodos tradicionales.
	Probabilidad: Muy baja con validación robusta.
	- Mitigación: Backup de sistema tradicional disponible.
	Validación en conjunto de test independiente.
	Monitoreo continuo de métricas en producción.

### Plan de contingencia
- Si accuracy cae <90% en producción → Alerta automática.
- Si accuracy cae <85% → Sistema se desactiva automáticamente.
- Revisión trimestral con datos reales de campo.

---

## 3. Fase 2: Obtención y Comprensión de Datos <a name="data_understanding"></a>

### El dataset para Recomendación de Cultivos

**Origen**: Conjunto de Datos para Recomendación de Cultivos  
**Tamaño**: 2,200 muestras  
**Granularidad**: Cada fila representa una muestra con el cultivo recomendado en función de la composición del suelo y las condiciones climáticas.

### ¿Por qué este dataset?

- **Clásico en ML**: Bien estudiado, podemos comparar resultados
- **Tamaño apropiado**: Ni muy pequeño ni muy grande para aprender
- **Múltiples tipos de datos**: Numéricos y categóricos

### Estrategia de descarga robusta

Implementaremos:
1. **Caché local**: Si ya descargamos, no repetir
2. **Manejo de errores**: Si falla la descarga, informar claramente
3. **Estructura organizada**: Carpeta datasets/ para todos los datos

In [ ]:
#importar funciones utilitarias implementads en el directorio src
import sys
sys.path.append('../src')

### Descarga y Carga de Datos

In [ ]:
# Función para descargar datos
import importlib
import data_loader
importlib.reload(data_loader)

from data_loader import load_crop_data

crop_data = load_crop_data()
print(crop_data.head())

### 🔍 Análisis Exploratorio de Datos (EDA)

#### ¿Qué es EDA y por qué es crucial?

**EDA es como ser un detective**: Buscas pistas, anomalías y patrones en los datos.

**John Tukey** (inventor del EDA) dijo: "Es mejor una respuesta aproximada a la pregunta correcta que una respuesta exacta a la pregunta incorrecta."

#### Primera impresión: Vista rápida

**¿Qué buscamos?**
- Tipos de datos (numéricos, texto, fechas)
- Dimensiones (filas × columnas)
- Valores faltantes obvios
- Rangos sospechosos

**Herramientas**: head(), info(), describe()

### Primera Inspección de Datos

In [ ]:
# Vista general del dataset
print("=" * 80)
print("INFORMACIÓN GENERAL DEL DATASET".center(80))
print("=" * 80)

# Mostrar primeras filas con formato mejorado
display(crop_data.head().style.background_gradient(cmap='coolwarm', subset=['temperature', 'humidity', 'ph', 'rainfall']))

# Información detallada
print("\n" + "=" * 80)
print("ESTRUCTURA DE DATOS".center(80))
print("=" * 80)
crop_data.info()

# Estadísticas descriptivas
print("\n" + "=" * 80)
print("ESTADÍSTICAS DESCRIPTIVAS".center(80))
print("=" * 80)
display(crop_data.describe().round(2).T)

### Descripción de Variables

### Entendiendo cada variable


In [ ]:
# Diccionario de metadatos
metadata = {
    'Variable': ['N','P','k','temperature','humidity','ph','rainfall','label'],
    'Tipo': ['Numérica', 'Numérica', 'Numérica', 'Numérica', 'Numérica', 
             'Numérica', 'Numérica', 'Categórica (target)'],
    'Descripción': [
        'Cantidad de nitrogeno en el suelo (en mg/kg)',
        'Cantidad de fosforo en el suelo (en mg/kg)',
        'Cantidad de potacio en el suelo (en mg/kg)',
        'Temperatura promedio (en °C)',
        'Porcentaje de humedad relativa (%)',
        'Valor del pH del suelo (0-14 escala cerca al cero más ácido)',
        'Precipitación (en mm)',
        'Tipo de cultivo recomendado para las condiciones dadas (ej: arroz, maíz, etc.)',
    ],
    'Valores Faltantes': [
        crop_data['N'].isnull().sum(),
        crop_data['P'].isnull().sum(),
        crop_data['K'].isnull().sum(),
        crop_data['temperature'].isnull().sum(),
        crop_data['humidity'].isnull().sum(),
        crop_data['ph'].isnull().sum(),
        crop_data['rainfall'].isnull().sum(),
        crop_data['label'].isnull().sum(),
    ]
}

df_metadata = pd.DataFrame(metadata)
display(df_metadata.style.map(
    lambda x: 'background-color: #ffcccc' if x > 0 else '', 
    subset=['Valores Faltantes']
))

---
### Detectando problemas en los datos

#### Análisis de Valores Faltantes (Missing values)


In [ ]:
# Análisis detallado de valores faltantes

from analyze_missing_values import analyze_missing_values

missing_analysis = analyze_missing_values(crop_data)
if missing_analysis is not None:
    print(missing_analysis)

### ✅ Análisis de Valores Faltantes

**Resumen:**  
- No se detectaron valores faltantes en ninguna columna.  
- Completitud global: **100%** de las celdas contienen datos válidos.

**Metodología aplicada**
1. Conteo absoluto por columna: `df.isnull().sum()`  
2. Porcentaje relativo: `(df.isnull().mean() * 100).round(2)`  
3. (Condicional) Visualizaciones: barra de % de faltantes y mapa de patrones (no generadas porque no hubo nulos).  
4. Criterios de alerta habituales:  
   - > 5%: monitoreo / imputación ligera  
   - > 5–20%: evaluar impacto y estrategia de imputación  
   - > 20%: considerar eliminación o técnicas avanzadas (MICE / modelos)  

**Resultados obtenidos**
| Métrica | Valor |
|---------|-------|
| Columnas con faltantes (>0) | 0 |
| Total de celdas nulas | 0 |
| Porcentaje global de faltantes | 0.00% |
| Columnas afectadas | Ninguna |
| Patrón estructurado de nulos | No aplica |

**Implicaciones prácticas**
- No se requiere imputación (media / mediana / moda / KNN / regresión).  
- No es necesario eliminar filas o columnas por incompletitud.  
- Menor riesgo de introducir sesgo por decisiones de relleno artificial.  
- Pipeline de preprocesamiento más simple (menos pasos y menos hiperparámetros).  
- Se acelera la experimentación en etapas posteriores (feature engineering y modelado).  

**Próximos pasos recomendados**
- Validar consistencia semántica (rango y plausibilidad de valores).  
- Monitorear valores atípicos y distribución (ya que no hay nulos, la calidad dependerá de outliers y escalas).  
- Asegurar que en futuras integraciones de datos (nuevas fuentes) se mantenga esta calidad mediante validaciones automáticas.

> Estado: Excelente calidad estructural respecto a completitud; la atención se puede enfocar ahora en variabilidad, outliers y relaciones entre variables.


### Análisis Univariado

#### 1. Variables Numéricas (7 características)

En este apartados analizaremos a cada una de las características (N, P, K, temperature, humidity, ph, rainfall)

- Estadísticas descriptivas completas
- Implicación de dispersión a través del coeficiente de variación
- Implicación de forma a través de la distribución de la gráfica (Asimetría y Curtosis)
- Confiabilidad de los datos (Validar valores Atípicos)

Finalmente tendremos un resumen ejecutivo que contenga toda esta información

In [ ]:
# Función para análisis univariado robusto

# Configuración para mostrar gráficos en Jupyter Notebook
%matplotlib inline

import importlib
import univariate_analysis
importlib.reload(univariate_analysis)

from univariate_analysis import analyze_numeric_values

# Variables numéricas
num_vars = ['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']
#  num_vars = ['temperature']

# Estadísticas descriptivas
print("\n" + "=" * 80)
print("ESTADÍSTICAS DESCRIPTIVAS".center(80))
print("=" * 80)
display(crop_data[num_vars].describe().round(2).T)

# Analizar variables numéricas clave
analyze_numeric_values(crop_data, num_vars)



#### Análisis de Entendimiento de Datos Univariado

Al ejecutar script de análisis univariado podemos observar que el dataset de recomendación de cultivos contiene **2,200 muestras** con **7 variables numéricas** que describen características del suelo y condiciones agroclimáticas. Todas las variables presentan **distribuciones NO normales** (p-value < 0.05), lo que indica la necesidad de técnicas robustas de preprocesamiento y modelado.

---

#### 1. Análisis por Variable

| Variable | Rango | Media | Mediana | Desv. Est. | CV | Asimetría | Curtosis | Outliers (%) | Características Clave |
|----------|-------|-------|---------|------------|-----|-----------|----------|--------------|----------------------|
| **Nitrógeno (N)** | 0-140 kg/ha | 50.55 | 37.00 | 36.92 | 73% | 0.51 | -1.06 | 0 (0%) | Alta variabilidad; asimetría positiva moderada; distribución aplastada; sin outliers |
| **Fósforo (P)** | 0-145 kg/ha | 53.36 | 51.00 | 32.99 | 62% | 1.01 | 0.86 | 138 (6.3%) | Asimetría positiva fuerte; cola derecha pronunciada; presencia significativa de outliers |
| **Potasio (K)** | 0-205 kg/ha | 48.15 | 32.00 | 50.65 | 105% | 2.38 | 4.45 | 200 (9.1%) | **Variable más problemática**; dispersión extrema; probable distribución bimodal; mayor proporción de outliers |
| **Temperatura** | 8.8-43.7°C | 25.62 | 25.60 | 5.06 | 20% | 0.18 | 1.23 | 86 (3.9%) | **Variable más balanceada**; casi simétrica; menor variabilidad relativa; bien comportada |
| **Humedad** | 14-100% | 71.48 | 80.47 | 22.26 | 31% | -1.09 | 0.30 | 30 (1.4%) | **Única con asimetría negativa**; mayoría de valores altos (70-100%); cola izquierda |
| **pH** | 3.5-9.9 | 6.47 | 6.43 | 0.77 | 12% | 0.28 | 1.66 | 57 (2.6%) | **Variable más concentrada**; menor CV; distribución equilibrada; valores concentrados en pH neutro |
| **Precipitación** | 20-298 mm | 103.46 | 94.87 | 54.96 | 53% | 0.97 | 0.61 | 100 (4.5%) | Alta variabilidad; asimetría positiva; discriminante para cultivos secano vs. trópico húmedo |

### 2. Hallazgos Transversales

#### 2.1 Normalidad
- **0 de 7 variables** siguen distribución normal (todas p-value < 0.05)
- **Implicación**: Modelos paramétricos tradicionales (LDA, Naive Bayes Gaussiano) pueden no ser óptimos
- **Recomendación**: Usar modelos tree-based (Random Forest, XGBoost) que no asumen normalidad

#### 2.2 Outliers
- Por el método de IQR, se evidencia que **6 de 7 variables** tienen outliers

#### 2. Variable Categórica (Objetivo)
- Análisis de 'label' (22 cultivos)
- Frecuencias y proporciones
- Balance de clases
- Gráficos de barras y pastel

### Análisis de Variable Categórica

In [ ]:
# Análisis de la variable categórica 'label' (cultivo recomendado)

# Configuración para mostrar gráficos en Jupyter Notebook
%matplotlib inline
import importlib
import analyze_categorical
importlib.reload(analyze_categorical)

from analyze_categorical import analyze_categorical

analyze_categorical(crop_data, 'label')

Nota: La **entropía** en el análisis de variables categóricas es una medida de incertidumbre o desorden en la distribución de las categorías. 

¿Qué significa en este contexto?
Si todas las clases tienen frecuencias similares, la entropía será alta (máxima diversidad, distribución uniforme).
Si una clase domina (por ejemplo, 90% de los datos), la entropía será baja (baja diversidad, alta concentración).

#### Matriz de correlación: Relaciones entre variables

**Correlación de Pearson**:
- Mide relación **lineal** entre variables
- Rango: [-1, +1]
- 0 = Sin relación lineal (¡pero puede haber no-lineal!)

**Interpretación**:
- |r| < 0.1: Muy débil
- 0.1 ≤ |r| < 0.3: Débil
- 0.3 ≤ |r| < 0.5: Moderada
- 0.5 ≤ |r| < 0.7: Fuerte
- |r| ≥ 0.7: Muy fuerte

**Cuidado**: Correlación ≠ Causalidad

In [ ]:
# Análisis de correlación mejorado
from correlation_analysis import correlation_analysis

correlation_analysis(crop_data)

### Interpretación de las Correlaciones

**Relación destacada y única:**
- P – K = **+0.736** → correlación muy fuerte y positiva. Sugiere posible multicolinealidad (nutrientes del suelo afectados por prácticas de fertilización similares o composición edáfica compartida).

**Relación secundaria (N – K):**
- Nitrogeno – Potasio muestra la segunda correlación más alta del conjunto pero permanece dentro del rango débil (|r| ≤ 0.231). Esto indica que NO hay una dependencia lineal fuerte entre ambos; podría reflejar prácticas de abonado combinadas o un gradiente edáfico suave, pero no alcanza niveles de preocupación por multicolinealidad. No requiere acciones de reducción (p.ej. eliminación, PCA) y en modelos lineales bastaría con regularización estándar.

**Resto de correlaciones:**
- Todas las demás están en rangos débiles o muy débiles (|r| ≤ 0.231) → baja redundancia lineal entre la mayoría de variables numéricas.

**Clima entre sí:**
- temperature – humidity = **+0.205** (débil). Plausible por co‐variación estacional / regional (más calor asociado a cierta retención de humedad en algunas zonas), pero no es determinante.

**Patrones suaves adicionales:**
- ph – rainfall = **−0.109**: ligera tendencia a menor pH (más ácido) con más lluvia; magnitud pequeña que sugiere efectos de lixiviación/acidificación leves.

**Conclusión práctica:**
- Salvo el binomio P–K, no existen dependencias lineales fuertes. Para un sistema de recomendación de cultivo (variable objetivo categórica) la señal predictiva probablemente provendrá de:
  1. Interacciones no lineales entre nutrientes y clima.
  2. Patrones multivariados más sutiles (combinaciones de N, P, K con pH y humedad).
  3. Posible importancia de variables individuales sin redundancia (baja colinealidad = modelo más estable).

**Implicación para modelado:**
- Modelos lineales podrían incluir regularización para manejar la multicolinealidad P–K.
- Árboles / ensembles (Random Forest, Gradient Boosting) capturarán interacciones sin requerir ingeniería pesada.
- Considerar verificar VIF solo para nutrientes si se usa un modelo lineal.

> En resumen: estructura de correlación limpia y parsimoniosa; no se requiere fuerte reducción por colinealidad, excepto vigilar el par P–K y monitorizar N–K si se añadieran nuevas fuentes de datos o más variables de suelo.

###Correlacion caracteristicas vs targe

### Detección de Anomalías y Outliers

**Outliers:** ¿Errores o información valiosa?

**Tipos de outliers**:
1. **Errores**:
2. **Casos raros pero válidos**: 
3. **Diferentes poblaciones**:

**Métodos de detección**:
- **IQR**: Fuera de Q1-1.5×IQR o Q3+1.5×IQR
- **Z-score**: |z| > 3
- **Isolation Forest**: Algoritmo de ML para anomalías

#### Distribuciones problemáticas

**Alta asimetría (skewness)**:
- Problema: Muchos algoritmos asumen normalidad
- Solución: Transformación log, sqrt o Box-Cox

**Alta curtosis**:
- Problema: Colas pesadas, muchos outliers
- Solución: Winsorization (cap de valores extremos)

In [ ]:
# Detección de outliers

# Configuración para mostrar gráficos en Jupyter Notebook
%matplotlib inline

import importlib
import detect_outliers
importlib.reload(detect_outliers)

from detect_outliers import detect_outliers

outliers_iqr, outliers_zscore, outliers_iso = detect_outliers(crop_data, False)

## 4. Fase 3: Preparación de los Datos

---

## 🎓 Material Adicional y Referencias

### Referencias Bibliográficas

In [ ]:
references = """
📚 REFERENCIAS:

[1] FAO (2017). The impact of crop choice on farm profitability. Food and Agriculture Organization. http://www.fao.org/3/i7658e/i7658e.pdf
[2] Bhola, A., & Kumar, P. (2024). Farm-Level Smart Crop Recommendation Framework Using Machine Learning. Annals of Data Science. https://link.springer.com/article/10.1007/s40745-024-00534-3
[3] Bakr, M. A., et al. (2025). Evaluation of Learning-Based Models for Crop Recommendation in Smart Agriculture. Information, MDPI. https://www.mdpi.com/2078-2489/16/8/632
[4] Van Klompenburg, T., Kassahun, A., & Catal, C. (2020). Crop yield prediction using machine learning: A systematic literature review. Computers and Electronics in Agriculture. https://doi.org/10.1016/j.compag.2020.105709
[5] Lal, R. (2015). Restoring Soil Quality to Mitigate Soil Degradation. Sustainability. https://doi.org/10.3390/su7055875
[6] Reddy, D. J., et al. (2024). Advancing crop recommendation system with supervised machine learning and explainable artificial intelligence. Scientific Reports. https://doi.org/10.1038/s41598-024-07003-8
"""

print(references)